# bio.tools querying

## Overview

## Code

In [74]:
import sys
import json
import requests
import argparse
import yaml
import biotools_parse as bp
import pandas as pd

# The call to get all bio.tools entries through the api is: https://bio.tools/api/tool/?format=json.
# Unfortunately, the response is limited to n tools. To get the whole set, the next pages must be retrieve.
# use "next" in the response to get succesive entries.
 
base_call = "https://bio.tools/api/tool/?format=json"


def make_request(URL):
    try:
        response = requests.get(URL)
    except:
        print('Could not make the request')
        return
    else:
        response = json.loads(response.text)
        return(response)

def build_url(next_page, filters):
    call_template = "https://bio.tools/api/tool/?{attributes}{next_page}&format=json"
    if next_page:
        next_page = "&%s"%next_page
    else:
        next_page = ""
    attributes = '&'.join(filters)    
    url = call_template.format(next_page=next_page, attributes=attributes)
    return(url)


def get_all_pages(filters):
    res = []
    next_page = "page=1"
    print('Starting making the requests...')
    while next_page:
        response = make_request(build_url(next_page, filters))
        print("Requesting: " + build_url(next_page, filters), end='\r')
        res = res + response["list"]
        next_page = response["next"]
        if next_page:
            next_page = next_page[1:]
    print('\nRequests finished.')
    return(res)

def save_result(out_path, result):
    with open(out_path, 'w') as out:
        json.dump(result, out)
    print('Result saved as ' + out_path )
    

colnames_general = ['name', 'description', 'type', 'topic', 'input', 'output']
colnames_detailed = [ 'name', 'description', 'version', 'type', 'topic', 'links', 'publication', 'download', 'inst_instr', 'test', 'src', 'os', 'input', 'output', 'dependencies', 'documentation', 'license', 'termsUse', 'contribPolicy', 'authors', 'repository']
def results_to_table(result, colnames):
    tools = bp.biotoolsToolsGenerator(result).instances
    # Load features into table
    colnames_features = colnames
    df_dict = {name : [] for name in colnames_features}

    for tool in tools:
        for field in colnames_features:
            df_dict[field].append(tool.__dict__.get(field))

    df_features = pd.DataFrame.from_dict(df_dict)
    return(df_features)


def parse_zooma_results(input_file):
    with open(input_file, 'r') as inp:
        zooma_terms_df = pd.read_csv(input_file)
        terms = zooma_terms_df['iri']
        terms = [term.split('http://edamontology.org/')[1] for term in terms if type(term) == str]
    return(terms)

def build_filter(term):
    filters_template =  {'topic': 'topicID="%s"', 'format':'dataFormatID="%s"', 'operation':'operationID="%s"', 'data':'dataTypeID="%s"'}
    if 'topic' in term:
        filters = filters_template['topic']%(term)
    elif 'data' in term:
        filters = filters_template['data']%(term)
    elif 'operation' in term:
        filters = filters_template['operation']%(term)
    elif 'format' in term:
        filters = filters_template['format']%(term)
    return(filters)     

def query_for_EDAM_terms(terms):
    '''
    Takes a list of EDAM terms and does a query for each
    Return a dictionary of results of the form: {<term>: <dataframe of results>}
    '''
    results = dict()
    results_detailed = dict()
    # Iteratre through terms
    for term in terms:
        # Avoid duplicated queries
        if term not in results.keys():
            # Building the filters using the EDAM terms
            filter_ = [build_filter(term)]
            # Do the query
            result = get_all_pages(filter_)
            # Put result in table
            result_df = results_to_table(result, colnames_general)
            result_detailed_df = results_to_table(result, colnames_detailed)
            # Put results table in dictionary with the results for the other terms
            results[term] = result_df
            results_detailed[term] = result_detailed_df
    return(results, results_detailed)

def count_tools_per_term(results):
    count_tools_per_term = {term :len(results[term]) for term in results.keys()}
    return(count_tools_per_term) 

def tools_per_term(results):
    tools_per_term = {term:results[term]['name'] for term in results}  
    return(tools_per_term)

def merge_tools_lists(results):
    tools_per_term_ = tools_per_term(results)
    tools = set()
    for term in tools_per_term_:
        for tool in tools_per_term_[term]:
            tools.add(tool)
    return(tools)

The attributes chosen to filter the query must be passed to the `get_all_pages` function in a list. Each attribute must be a string of the form `name=signalp`. This string will be put as a parameter in the query URL as it is. Please, follow the [bio.tools API reference](https://biotools.readthedocs.io/en/latest/api_reference.html) to construct this parameters successfully.

### Simple Query Example 1:

The following is an example of a query of tools annotted with the EDAM topic "Sequence composition, complexity and repeats" and with the word "game"  in their description. 

In [3]:
sample_filters = ['topic="Sequence composition, complexity and repeats"', 'description="game"']
filters = sample_filters
result = get_all_pages(filters)

Starting making the requests...
Requesting: https://bio.tools/api/tool/?topic="Sequence composition, complexity and repeats"&description="game"&page=1&format=json
Requests finished.


In [4]:
print(result)

[{'elixirCommunity': [], 'collectionID': ['EMBOSS'], 'accessibility': None, 'community': None, 'topic': [{'term': 'Sequence composition, complexity and repeats', 'uri': 'http://edamontology.org/topic_0157'}], 'owner': 'EMBOSS', 'cost': 'Free of charge', 'relation': [], 'download': [{'url': 'http://emboss.open-bio.org/html/adm/ch01s01.html', 'note': None, 'version': None, 'type': 'Source code'}, {'url': 'http://emboss.open-bio.org/html/adm/ch01s01.html', 'note': None, 'version': None, 'type': 'Binaries'}], 'validated': 1, 'publication': [{'doi': '10.1016/S0168-9525(00)02024-2', 'note': None, 'version': None, 'pmid': None, 'type': ['Primary'], 'pmcid': None, 'metadata': {'title': 'EMBOSS: The European Molecular Biology Open Software Suite', 'abstract': '', 'citationCount': 4828, 'authors': [{'name': 'Rice P.'}, {'name': 'Longden L.'}, {'name': 'Bleasby A.'}], 'date': '2000-06-01T00:00:00Z', 'journal': 'Trends in Genetics'}}, {'doi': '10.1017/CBO9781139151399', 'note': None, 'version': No

To save the rusult, use the `save_result(out_path, result)` funtion. `out_path` is the path where the `result` will be saved.

In [8]:
save_result('run_3.json', result)

Result saved as run_3.json


In [9]:
df_feaures = results_to_table(result)

###  Simple Query Example 2:

In [10]:
sample_filters = ['operation="Formatting"']
filters = sample_filters
result = get_all_pages(filters)

print("Number of tools found: %d"%len(result))

Starting making the requests...
Requesting: https://bio.tools/api/tool/?operation="Formatting"&page=39&format=json
Requests finished.
Number of tools found: 387


## Case 1: ETL tools discovery

We will query bio.tools using a set of EDAM terms that have being obtained after a mapping with ZOOMA.


### Step 1: parsing zooma results

ZOOMA results must be in a csv format.The colunm names, in this order, must be: `keyword`,`iri`,`label` and `confidence`. An example of a valid: 

|keyword            |iri                                   |label                      |confidence|
|-------------------|--------------------------------------|---------------------------|----------|
|Ontology annotation|http://edamontology.org/operation_0226|Annotation                 |MEDIUM    |
|Ontology annotation|                                      |                           |          |
|semantic annotation|                                      |                           |          |
|semantic annotation|http://edamontology.org/operation_3778|Text annotation            |MEDIUM    |
|Text mining        |http://edamontology.org/topic_0218    |Natural language processing|GOOD      |

Raw: 

```
keyword,iri,label,confidence
Ontology annotation,http://edamontology.org/operation_0226,Annotation,MEDIUM
Ontology annotation,,,
semantic annotation,,,
semantic annotation,http://edamontology.org/operation_3778,Text annotation,MEDIUM
Text mining,http://edamontology.org/topic_0218,Natural language processing,GOOD
```

The function `parse_zooma_results` parses this csv and returns a list of terms suitable for the querying.

For a mapping of keyword related with ETL against EDAM, we got the ZOOMA results in the path `ETL_EDAM_curated.csv`.

In [39]:
terms_file='ETL_EDAM_curated.csv'
ETL_edam_terms = parse_zooma_results(terms_file)
print(ETL_edam_terms)

['data_2337', 'data_0949', 'operation_3907', 'topic_3489', 'data_2337', 'topic_3572', 'operation_0336', 'operation_0335', 'operation_3434', 'operation_1812', 'data_0842', 'data_0006', 'topic_3366']


### Step 2: doing the query. 

The function `query_for_EDAM_terms` does the quering for the previously obtained list of EDAM terms. It resturns two kind of outputs: general and detailed. Each is a dictionary of the form: `{<term>: <dataframe of results>}`

In [52]:
ETL_results_general, ETL_results_detailed  = query_for_EDAM_terms(ETL_edam_terms)

Starting making the requests...
Requesting: https://bio.tools/api/tool/?dataTypeID="data_2337"&page=2&format=json
Requests finished.
Starting making the requests...
Requesting: https://bio.tools/api/tool/?dataTypeID="data_0949"&page=1&format=json
Requests finished.
Starting making the requests...
Requesting: https://bio.tools/api/tool/?operationID="operation_3907"&page=1&format=json
Requests finished.
Starting making the requests...
Requesting: https://bio.tools/api/tool/?topicID="topic_3489"&page=19&format=json
Requests finished.
Starting making the requests...
Requesting: https://bio.tools/api/tool/?topicID="topic_3572"&page=18&format=json
Requests finished.
Starting making the requests...
Requesting: https://bio.tools/api/tool/?operationID="operation_0336"&page=2&format=json
Requests finished.
Starting making the requests...
Requesting: https://bio.tools/api/tool/?operationID="operation_0335"&page=36&format=json
Requests finished.
Starting making the requests...
Requesting: https://

#### Total number of terms

In [56]:
len(ETL_results_general.keys())

12

#### Tools per term

In [67]:
count_tools_per_term(ETL_results_general)

{'data_0006': 344,
 'data_0842': 86,
 'data_0949': 4,
 'data_2337': 14,
 'operation_0335': 364,
 'operation_0336': 23,
 'operation_1812': 210,
 'operation_3434': 35,
 'operation_3907': 6,
 'topic_3366': 79,
 'topic_3489': 187,
 'topic_3572': 181}

#### Lists of tool names per term
This is handy for validation

In [72]:
tools_per_term(ETL_results_general)

{'data_0006': 0                       fconverter
 1                            scope
 2            gatk_haplotype_caller
 3        elixir elearning platform
 4                          phaseme
                   ...             
 339                       osdb api
 340                   biojs-io-gff
 341              biojs-vis-feature
 342                  biojs-vis-hgv
 343    imgt hla sequence alignment
 Name: name, Length: 344, dtype: object,
 'data_0842': 0                                              disgenet
 1                                              disgenet
 2                                               faidare
 3                                                pbg-ld
 4                                                pbg-ld
                             ...                        
 81                                                 evex
 82                                         plaza dicots
 83           database of genes from ncbi refseq genomes
 84    bindingdb databas

#### Merged tools lists
All tools discovered merged in a single set

In [78]:
all_ETL_tools = merge_tools_lists(ETL_results_general)
all_ETL_tools

{'restrictfromsize',
 'goslin',
 '.net bio',
 'uniprotr',
 'evidence finder',
 'presartools',
 'imex',
 'zlibbioc',
 'genscalpel',
 'prophecy',
 'pgxmine',
 'complex portal',
 'gaggle',
 'fish - profile search',
 'laj_1',
 'mtbgt',
 'sbmlmod',
 'prosextract',
 'amelie',
 'bcm search launcher sequence utilities',
 'pyrates',
 'removeexonlines',
 'annotationforge',
 'genatlas',
 'ngsutils',
 'nanoj-squirrel',
 'rightfield',
 'mowserv',
 'xml2raf 1.02 -create raf maps from pdb xml files',
 'bsa4yeast',
 'nrg-cing',
 'adjustpkl',
 'mappinghsp4taxo',
 'got-10k',
 'nextgen utility scripts',
 'aggregate_scores_in_intervals',
 'genbank2gff3',
 'aceparser',
 'haplotyper',
 'biojs-io-graduates',
 'deseq',
 'snpeff',
 'imgt ligm-db',
 'hgv_beam',
 'rre',
 'pride toolsuite',
 'keggtranslator',
 'foldamerdb',
 'plotgenomecoverage',
 'gdc',
 'mimic-cxr',
 'pfind studio',
 'gdtools_simulate-mutations',
 'getnumber',
 'cregis-q',
 'maketssdist',
 'fastqvalidator',
 'glycan array dashboard',
 'dgva',
 

In [79]:
len(all_ETL_tools)

1307

## Case 2: Ontology tools discovery

In this case, the file of zooma results is `ontology_annotation_EDAM_curated_csv.csv`.

### Step 1: parsing zooma results

In [37]:
terms_file='ontology_annotation_EDAM_curated_csv.csv'
ontology_edam_terms = parse_zooma_results(terms_file)
print(ontology_edam_terms)

['operation_0226', 'operation_3778', 'topic_0089', 'operation_3280']


### Step 2: doing the query. 

In [38]:
ontology_results_general, ontology_results_detailed  = query_for_EDAM_terms(ontology_edam_terms)

Starting making the requests...
Requesting: https://bio.tools/api/tool/?operationID="operation_0226"&page=14&format=json
Requests finished.
Starting making the requests...
Requesting: https://bio.tools/api/tool/?operationID="operation_3778"&page=5&format=json
Requests finished.
Starting making the requests...
Requesting: https://bio.tools/api/tool/?topicID="topic_0089"&page=29&format=json
Requests finished.
Starting making the requests...
Requesting: https://bio.tools/api/tool/?operationID="operation_3280"&page=6&format=json
Requests finished.


#### Total number of terms

In [57]:
len(ontology_results_general.keys())

4

#### Tools per term

In [61]:
count_tools_per_term(ontology_results_general)

{'operation_0226': 145,
 'operation_3280': 61,
 'operation_3778': 48,
 'topic_0089': 305}

#### Lists of tool names per term
This is handy for validation

In [80]:
tools_per_term(ontology_results_general)

{'operation_0226': 0                     treeio
 1      workflow4metabolomics
 2                globalfungi
 3                      amtdb
 4              causalbuilder
                ...          
 140               metdisease
 141                     magi
 142                    npact
 143                     cogs
 144                    marky
 Name: name, Length: 145, dtype: object, 'operation_3280': 0         dquest
 1        ontokin
 2         nlimed
 3           vist
 4       eventepi
          ...    
 56         acela
 57          deca
 58    facta plus
 59    simconcept
 60        bagel2
 Name: name, Length: 61, dtype: object, 'operation_3778': 0                                   webanno
 1                        monarch initiative
 2                            mesophotic.org
 3               medical relation extraction
 4                                  snpnotes
 5                                 civicmine
 6                                     koffi
 7                      

#### Merged tools lists
All tools discovered merged in a single set

In [81]:
all_ontology_tools = merge_tools_lists(ontology_results_general)
all_ontology_tools

{'a4preproc',
 'aber-owl',
 'acela',
 'aceparser',
 'ads',
 'agrigo',
 'ahtpdb',
 'aiso',
 'ajia',
 'alliance genome resources',
 'amigo 2',
 'amtdb',
 'anabel',
 'aniseed',
 'annot',
 'annotationfilter',
 'annotsv',
 'apache taverna',
 'arachnoserver',
 'aristo',
 'asa-ps',
 'aspgd',
 'association viewer',
 'bagel2',
 'becas',
 'beere',
 'bel commons',
 'bel2abm',
 'belief',
 'bgeedb',
 'bio.tools',
 'bio2bel',
 'biobert',
 'biocaddie',
 'biocatalogue',
 'bioecs',
 'bioexcel genome sequence quality control workflow',
 'biogateway',
 'biojs-vis-gprofiler',
 'biolabeler',
 'biolit',
 'biomoby',
 'biomodels database',
 'bionda - biomarker database',
 'bioner',
 'bionet-mining',
 'bioportal',
 'bior',
 'bittersweet',
 'biwalklda',
 'bo-lstm',
 'brisskit',
 'bubastis',
 'bugbuilder',
 'cagefightr',
 'calchas',
 'candida genome database cgd',
 'card',
 'cargo',
 'carmo',
 'causalbuilder',
 'caw',
 'cbo',
 'cddapp',
 'cdfread',
 'cedar ondemand',
 'cellbaser',
 'ceu mass mediator',
 'charpro

In [82]:
len(all_ontology_tools)

476